# Graph Machine Learning Introduction

by Alejandro Correa Bahnsen, Jaime D. Acevedo-Viloria & Luisa Roa

version 1.2, October 2022

In this notebook we will be doing a brief introduction to graph machine learning. The agenda is as follows:


1. Different types of graphs - Introduction to NetworkX
2. Creating Graph Based Features and enhancing ML models
3. Creating a Graph from own data using NetworkX
4. Transductive Learning vs. Inductive Learning
5. Graph Neural Networks - Introduction to DGL

Through these basics you will be able to leverage graphs for the enhacement of Machine Learning models, or be able to learn the basics of how to build Neural Networks specially crafted for Graphs. We hope you like it!

In [ ]:
#Install libraries
!pip install dgl==0.6.1
!pip install torch==1.9.0

## Types of Graphs - An Introduction to NetworkX

NetworkX according to it's creators is: NetworkX is a Python package for the creation, manipulation, and study of the structure, dynamics, and functions of complex networks. 

![](https://raw.githubusercontent.com/jdacevedo3010/graph-mahine-learning-workshop/master/images/networkx_description.png)

https://networkx.org/

We will be using the latest stable 2.8.7 version of the Package as referenced on the requirements .txt provided in the repo.

In [ ]:
import networkx as nx
import numpy as np
import pandas as pd

Let's start by describing different properties graphs can have and what those mean for the graph in subject. We will use NetworkX visual examples for every one of them and we will also describe real world applications where you may find such type of graph.

We will be using different NetworkX to innitialize graphs, this is just to highlight the many different ways we can do this. Make sure to check the documentation for more info in this: https://networkx.org/documentation/stable/reference/generators.html

### Directed & Undirected Graphs

This property refers as to whether the edges connecting the graphs have an inherent direction in it.

In undirected the graphs, edges indicate a two-way relationship, and as such they can be traversed from either node to other connected. In directed graphs, edges indicate a one-way direction. Meaning that they can only be traversed in an specific direction of the edge.

In [ ]:
#Initilize a random graph
g_gaussian = nx.gaussian_random_partition_graph(25, 4, 5, 0.25, 0.1, directed=False)

#Draw the graph structure
nx.draw(g_gaussian, with_labels=True)

In [ ]:
#Initialize a random directed graph
g_gaussian_dir = nx.gaussian_random_partition_graph(25, 4, 5, 0.25, 0.1, directed=True, seed=23)

#Draw the graph structure
nx.draw(g_gaussian_dir, with_labels=True)

This is normally described by the use of an arrow pointing the direction of the edge, if there is no arrow then we can assume the graph is undirected. We can see this in the above networkX examples, where we create both a undirected and directed Random Gaussian Graph setting the directed parameter to True and False respectively.

This is a very important characteristic in real life application. When creating graphs to describe real-life processes we might have an Instagram like Social Network, where an user can follow another user but it doesn't have to be the other way around. This would be a directed graph.

We may also have a Facebook like Social Network, where when a friend request is sent and accepted both users are instantly connected to each other. This could be described by an undirected graph.

We can also see this, when extracting the edges of the graph with the edges() method, take a look:

In [ ]:
g_gaussian.edges()

In [ ]:
g_gaussian_dir.edges()

This accesses the EdgeView for the undirected graphs where all edges are given as undirected. While, on the directed graph we can see that it defaults to the OutEdges of the graph, meaning that those tuples represent just a one-way path from one node to the other.

### Weighted and Unweighted

Another property that graphs can have is that they have edges with different weights, this means that in the relationship between two nodes there is a numerical value that indicates the magnitude of how strong or important the relationship is. In this way, weighted graphs are those that contain a weight on each edge while a unweighted graph does not.

In [ ]:
import random
import matplotlib.pyplot as plt

#Inicialize a random graph
G=nx.gnm_random_graph(35, 50, seed=12)

#Assign a random weights
for (u, v) in G.edges():
    G.edges[u,v]['weight'] = random.random()

#Define type of relationship
family = [(u, v) for (u, v, d) in G.edges(data=True) if d["weight"] >= 0.6]
friend = [(u, v) for (u, v, d) in G.edges(data=True) if d["weight"] < 0.6]

#Plot
pos = nx.spring_layout(G, seed=124)
nx.draw_networkx_nodes(G, pos)
nx.draw_networkx_edges(
    G, pos, edgelist=family, width=2, edge_color='r', alpha = 0.4
)
nx.draw_networkx_edges(
    G, pos, edgelist=friend, width=1, edge_color='b', alpha = 0.4
)

nx.draw_networkx_labels(G, pos)


ax = plt.gca()
ax.margins(0.08)
plt.axis("off")
plt.tight_layout()
plt.show()

We create a graph with 35 nodes and 50 random edges with the gnm_random_graph function, then we assign the property 'weights' to the edges with a random probability. Lets now assume that it is a social network, where each node is a person who has different connections with friends or family, if the weight is greater than 0.6 then there is a closer connection so they are family, but if the weight is less than 0.6 then the relationship is less strong and they are friends.

Although this is a simple example, in many real-world applications it is an important characteristic to consider since not all relatinoships are equal. Furthermore, different graphs algorithms are affected, for example in the shortest path algorith what used to be the shortest path can also be the most expensive given the weights in the edges.

### Sparse & Dense Graphs

This is an interesting distinction, the density of the graph refers to both the number of nodes and how connected are them. In mathematics, a dense graph is a graph in which the number of edges is close to the maximal number of edges. The maximal number of edges being all the pair combination of nodes in an undirected graph, and all the pair permutation of nodes in an undirected graph.

In other words, the more close to fully connected the graph is - fully connected meaning that every node is connected to each other-, the more dense we can say the graph is. If the graph is a bunch of disjunct islands of nodes, we can safely say that is a sparse graph. Let's see some networkX examples:

In [ ]:
#Set number of nodes n
n = 25

#Initialize the sparse graph
g_sparse = nx.erdos_renyi_graph(n, 0.08,seed=23)

#Draw the graph structure
nx.draw(g_sparse, with_labels=True)

In [ ]:
#Initialize the dense graph
g_dense = nx.erdos_renyi_graph(n, 0.15,seed=23)

#Draw the graph structure
nx.draw(g_dense, with_labels=True)

We create the graphs using the generator method erdos_renyi_graph, this is a commonly used model to generate random graphs from two parameters: The number of nodes n, and the probability to create an edge for every pair of nodes p.

Therefore, for the sparse graph we set up a small probability of 0.08, and for the dense graph we set up a probability of 0.15.

We can also measure the Density of the graph, as the ratio of the number of edges with respect to the maximal number of edges. Therefore we can use the following formulas for undirected and directed graphs:

Undirected: ![](https://raw.githubusercontent.com/jdacevedo3010/graph-mahine-learning-workshop/master/images/undirected.png)

Directed: ![](https://raw.githubusercontent.com/jdacevedo3010/graph-mahine-learning-workshop/master/images/directed.png)

Now, let's measure the density of the previously created graphs!

First we need the number of edges for both graphs:

In [ ]:
e_sparse = g_sparse.number_of_edges()
e_dense = g_dense.number_of_edges()

print("The number of edges for the sparse graph is: ", e_sparse)
print("The number of edges for the dense graph is: ", e_dense)

As you can see we used the number_of_edges() method to measure how many edges we have in each graph automatically. Here we can see other properties we can take from NetworkX graphs: https://networkx.org/documentation/stable/reference/classes/graph.html.

Like for example, wether a certain node or edge exists.

Now try to recreate the formula for undirected graphs in the following function:

In [ ]:
def calculate_undirected_graph_density(g):
    '''
    Hint you should use the Graph Class attributes to get
    the number of nodes and Edges
    '''
    n_nodes = 0 #Your code goes here instead of the zero
    n_edges = 0 #Your code goes here instead of the zero
    density = 0 #Your code goes here instead of the zero
    
    return density

In [ ]:
density_sparse = calculate_undirected_graph_density(g_sparse)
density_dense = calculate_undirected_graph_density(g_dense)

print("The density of the sparse graph is: ",round(density_sparse,2))
print("The density of the dense graph is: ",round(density_dense,2))

Now let's try the directed graph density:

In [ ]:
def calculate_directed_graph_density(g):
    '''
    Hint you should use the Graph Class attributes to get
    the number of nodes and Edges
    '''
    n_nodes = 0 #Your code goes here instead of the zero
    n_edges = 0 #Your code goes here instead of the zero
    density = 0 #Your code goes here instead of the zero
    
    return density

In [ ]:
density_directed = calculate_directed_graph_density(g_gaussian_dir)

print("The density of the directed graph is: ",round(density_directed,2))

In real-life, determining whether a graph is dense or sparse is quite subjective to the problem at hand. For example, we can use this metric to compare two different social networks and then compare one with each other in terms of density. Another highly used strategy to effectively measure the density of a given graph is via the comparison to multiple generated random graphs.

### Homogeneous and Heterogeneous Graphs

Last, but definitely not least, we have Homogeneous and Heterogeneous Graphs. This distinction goes into the types of both the nodes and the edges in the graph. In cases where we have only one type of node and only one type of relationship we are dealing with an Homogeneous Graph, any othe type of node or edge added would then be a Heterogeneous Graph.

This is a really important distinction, because those graph are inmensely different both in their complexity -where Homogeneous Graphs tend to be much simpler-, and in the Machine Learning methodologies we can use to deal with them that we will study in later posts.

In [ ]:
import random

#Inicialize a random graph
G=nx.erdos_renyi_graph(n, 0.15,seed=45)

#Assign type of node
for u in G.nodes():
    rand = random.randint(0, 48)
    G.nodes[u]['Type'] = 1 if rand < 12 else 0

#Draw the graph with specified colors for each node type
color_val = [nx.get_node_attributes(G,'Type').get(node) for node in G.nodes()]
nx.draw(G, pos, with_labels=True, node_color = color_val)

In this case, we construct a random graph with 25 nodes and a probability of 0.15 of having an edge between nodes. For our example, we consider a setting of a communications company where the graph considers calls between users. Particularly, the company wants to differentiate between new users (less than 12 months) and old users (more than 12 months) to understand if there are different behaviors between users.

As in our example, most real-life graphs are heterogeneous graphs where different entities interact. It is quite complicated to describe a process with only one type of entity interacting with other entities all of the same type, making Heterogenous Graphs more challenging to deal with but also more expressive of the process they describe.

It's also relevant to note that different type of nodes may have different characteristics or features. Think of a paper-author network, where we have author relationships between Author and Paper nodes, and citation relationships where Paper nodes connect with each other if they have cited any other paper. In this type of example, we may have really distinct feature sets between the Authors and the Papers, and as such we have to be more creative when dealing with those networks.

NetworkX also has a number of pre-determined made Graphs, such as this one with the coappearance network of characters in the novel Les Miserables.

In [ ]:
lm_graph = nx.les_miserables_graph()
nx.draw(lm_graph, with_labels=True)

characters = lm_graph.nodes()

Now let's try to get the neighbors of the character Myriel using a built-in function of the NetworkX Graph Class:

In [ ]:
# Hint you should transform the output of the built-in function into a list to show the results
# Your code goes here

Exercise:

With any of the Graph Generators (https://networkx.org/documentation/stable/reference/generators.html) Create a graph of preference, and through a built-in method of the NetworkX Graph Class get the neighbors of a certain node:


In [ ]:
# Your code Goes here

## Creating Graph Based Features and enhancing ML models 

Now let's see how we can use these new features taken from the graph to enhance our ML models. First, let's import the information of the users in the graph from the nodes_features_workshop csv.

In [ ]:
df = pd.read_csv('https://github.com/jdacevedo3010/graph-mahine-learning-workshop/raw/master/data/nodes_features_workshop.csv').set_index('USER_ID')
df.head()

Here we have a DataFrame with the users as the index. The columns contain the features that profile them:

1. Device Type: An encoding of the different devices in the dataset
2. Expected Value: A score that measures the value the client will bring
3. Sales: Total amount spent by the user

And a label that tells us whether the user is fraudulent or not (FRAUD column).

Let's use this information to train a couple of traditional Machine Learning models such as: Gradient Boosting Trees, and Logistic Regression. Let's first create train and test masks, we will do this manually given that we wil later need this same division for the Graph Neural Networks. We will also be using torch tensors for the same reason.

In [ ]:
import torch as th

def create_masks(df, seed=23, test_size=0.2):
    '''
    This function creates binary tensors that indicate whether an user is on the train or test set
    '''
    np.random.RandomState(seed)
    temp_df = df.copy()
    temp_df['split_flag'] = np.random.random(df.shape[0])
    train_mask = th.BoolTensor(np.where(temp_df['split_flag'] <= (1 - test_size), True, False))
    test_mask = th.BoolTensor(np.where((1 - test_size) < temp_df['split_flag'] , True, False))
    return train_mask, test_mask

In [ ]:
#Create binary masks
train_mask, test_mask = create_masks(df, 23, 0.3)

print(train_mask)

#Here we transform the tensors so they indicate the indices of the train and test users instead of the binary
train_nid = train_mask.nonzero().squeeze()
test_nid = test_mask.nonzero().squeeze()

print(train_nid)

Now, let's create the X and Y tensors:

In [ ]:
#Create X and Y dataframes
X = df.drop(['FRAUD'], axis=1)
y = df.drop(['DEVICE_TYPE','EXPECTED_VALUE','SALES'], axis=1)

print('The shape of the X DataFrame is: ',X.shape)
print('The shape of the y DataFrame is: ',y.shape)

In [ ]:
#Transform the X and Y dataframes to tensors now as well
X = th.tensor(X.values).float()
y = th.tensor(y.values).type(th.LongTensor).squeeze_()

print(X.shape)
print(y.shape)

Let's create the functions to train the ML models, and a function that allows us to measure the performance of those models in terms of ROC Curve AUC, F1-Score, Precision and Recall:

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score

def get_gb_preds(X_train, y_train, X_test, seed=23):
    clf = GradientBoostingClassifier(random_state=seed)
    clf.fit(X_train,y_train)
    y_pred_probas = clf.predict_proba(X_test)
    return y_pred_probas

def get_lr_preds(X_train, y_train, X_test, seed=23):
    clf = LogisticRegression(random_state=seed)
    clf.fit(X_train,y_train)
    y_pred_probas = clf.predict_proba(X_test)
    return y_pred_probas

def get_results(y_pred_probas, y_test, threshold=0.5):
    pred_probas_1 = y_pred_probas[:,1]
    preds_1 = np.where(pred_probas_1>threshold,1,0)
    auc = roc_auc_score(y_test, pred_probas_1)
    f1 = f1_score(y_test,preds_1)
    prec = precision_score(y_test,preds_1)
    recall = recall_score(y_test,preds_1)
    return auc, f1, prec, recall

#### Logistic Regression Results

In [ ]:
X[train_nid].shape

In [ ]:
model = 'logistic-regression'
y_pred_probas = get_lr_preds(X[train_nid], y[train_nid], X[test_nid], seed=23)

results_df = pd.DataFrame(columns=['Model','AUC','F1 Score','Precision','Recall'])
auc, f1, prec, recall = get_results(y_pred_probas, y[test_nid], 0.5)
dict_results = {'Model':model, 'AUC':auc, 'F1 Score':f1, 'Precision':prec, 'Recall':recall}
results_df = results_df.append(dict_results, ignore_index=True)

results_df

#### GBoost results

In [ ]:
model = 'GBoost'
y_pred_probas = get_gb_preds(X[train_nid], y[train_nid], X[test_nid], seed=23)

auc, f1, prec, recall = get_results(y_pred_probas, y[test_nid], 0.5)
dict_results = {'Model':model, 'AUC':auc, 'F1 Score':f1, 'Precision':prec, 'Recall':recall}
results_df = results_df.append(dict_results, ignore_index=True)

results_df

## Creating a Graph from own data using NetworkX

We will know see how to create a graph from data instead of randomly. For this we will have to import the csv's in the data folder and process them for NetworkX.

First lets import the relations csv in the data folder:

In [ ]:
import pandas as pd

edges_df = pd.read_csv('https://github.com/jdacevedo3010/graph-mahine-learning-workshop/raw/master/data/new_edges_workshop.csv')
edges_df.head()

In [ ]:
edges_df.shape

Here you a 2-colummn DataFrame that contains the undirected edges between distinct users. This is normally referred to as "List of edges" and it's a common way to create graphs. NetworkX also has a method to create a graph from a DataFrame of edges, let's do just that:

In [ ]:
G = nx.from_pandas_edgelist(edges_df,'~from','~to')
G.number_of_nodes()

Let's draw a portion of the graph to check it out:

In [ ]:
G_draw = nx.from_pandas_edgelist(edges_df.head(100),'~from','~to')
nx.draw(G_draw)

Nice! We can now use this created graph to extract characteristics from it. 

Let's say we want to get the number of neighbors of each node in the graph:

In [ ]:
degrees = dict(G.degree)

We can use the degree method in NetworkX to create a dictionary that holds the node id's as the keys and the degree of that node as the value. There are also plenty of other measures, we can extract from the Graph object of NetworkX like centrality or betweeness metrics.

More information about the NetworkX library can be found in this tutorial: https://networkx.org/nx-guides/content/tutorial.html. Or in the overall documentation guide of the package: https://networkx.org/nx-guides/index.html

#### And now let's enhance the features with some extracted from the graphs

Now we will use the previously generated dictionary of degrees as an additional feature to the DataFrame, along with the centrality measure PageRank.

PageRank was developed by Google and measures the importance of a node in a Graph given how connected are the node's neighbors. More information can be found here: https://es.wikipedia.org/wiki/PageRank

Let's first calculate that for the previously generated graph:

In [ ]:
pr = nx.pagerank(G,alpha=0.9)


Let's take a look at why those values for two nodes are so different. First, we will select a high PageRank node and a low PageRank node. Then, we need to get the 2-hop neighbors of those nodes in a list using the neighbors() method (https://networkx.org/documentation/stable/reference/classes/generated/networkx.Graph.neighbors.html):

In [ ]:
print('The PageRank of the selected highly central node is: ',pr[38709])
print('The PageRank of the selected lowly central node is: ',pr[125388])

In [ ]:
def flatten_2d_list(matrix):
    flatten_matrix = []
    for sublist in matrix:
        for val in sublist:
            flatten_matrix.append(val)
    return flatten_matrix

high_neighbors = [n for n in G.neighbors(38709)]
high_neighbors_2 = flatten_2d_list([[n for n in G.neighbors(n2)] for n2 in high_neighbors])
low_neighbors = [n for n in G.neighbors(125388)]
low_neighbors_2 = flatten_2d_list([[n for n in G.neighbors(n2)] for n2 in low_neighbors])


print("Number of 2-hop neighbors of high PageRank: ",len(high_neighbors_2))
print("Number of 2-hop neighbors of low PageRank: ", len(low_neighbors))

As mentioned before, the PageRank measures the centrality of a node given the connections of his neighbors. So in a social network, if my friends have many friends then I'll be a highly central node.

Now let's draw those subgraphs using the subgraph method of networkx (https://networkx.org/documentation/stable/reference/classes/generated/networkx.Graph.subgraph.html) for a better look:

In [ ]:
nx.draw(G.subgraph(high_neighbors_2))

In [ ]:
nx.draw(G.subgraph(low_neighbors_2))

We can see that the highly central node has every node highly interconnected with each other, while the low PageRank node has a cluster in the middle and the a satelite circunference around that.

And now let's add both the degree and PageRank as features for the users:

In [ ]:
#Map Degree and PageRank into the DataFrame
df_enhanced = df.copy()
df_enhanced['DEGREE'] = df.index.map(degrees)
df_enhanced['PAGERANK'] = df.index.map(pr)

df_enhanced.head()

And finally, let's run the same models as before with these new features to see how the results compare with each other:

In [ ]:
#Create X and Y dataframes
X_enhanced = df_enhanced.drop(['FRAUD'], axis=1).fillna(0)
y_enhanced = df_enhanced.drop(['DEVICE_TYPE','EXPECTED_VALUE','SALES','DEGREE','PAGERANK'], axis=1)

print('The shape of the X DataFrame is: ',X.shape)
print('The shape of the y DataFrame is: ',y.shape)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

X_enhanced[['DEGREE','PAGERANK']] = scaler.fit_transform(X_enhanced[['DEGREE','PAGERANK']])

In [ ]:
th.tensor(X_enhanced.values).float().shape

In [ ]:
#Transform the X and Y dataframes to tensors now as well
X_enhanced = th.tensor(X_enhanced.values).float()
y_enhanced = th.tensor(y_enhanced.values).type(th.LongTensor).squeeze_()

print(X_enhanced.shape)
print(y_enhanced.shape)

In [ ]:
model = 'logistic-regression-enhanced'
y_pred_probas = get_lr_preds(X_enhanced[train_nid], y_enhanced[train_nid], X_enhanced[test_nid], seed=23)

auc, f1, prec, recall = get_results(y_pred_probas, y_enhanced[test_nid], 0.5)
dict_results = {'Model':model, 'AUC':auc, 'F1 Score':f1, 'Precision':prec, 'Recall':recall}
results_df = results_df.append(dict_results, ignore_index=True)

results_df

In [ ]:
model = 'GBoost-enhanced'
y_pred_probas = get_gb_preds(X_enhanced[train_nid], y_enhanced[train_nid], X_enhanced[test_nid], seed=23)

auc, f1, prec, recall = get_results(y_pred_probas, y_enhanced[test_nid], 0.5)
dict_results = {'Model':model, 'AUC':auc, 'F1 Score':f1, 'Precision':prec, 'Recall':recall}
results_df = results_df.append(dict_results, ignore_index=True)

results_df

It looks like in this case the new added features from the graph are not achieving a better performance for the Machine Learning model, this could be due to the model already doind a pretty good job with the non-graph features and because of the social graph not provding usefull information for fraude detection that makes sense given that fraudulents probably would try to hide from connections. Maybe another type of graph can be better to this task.

Given the rarity of these Graph-Based Features they carry vastly different information from the tipically used features and as such allow the model to better differentiate classes, that can lead to better performance in models.This conclusions is further developed on our previously published paper: Supporting Financial Inclusion with Graph Machine Learning and Super-App Alternative Data; where we prove how graph based features augments the AUC of Credit RIsk models up to 4-5 percentage points!

https://arxiv.org/abs/2102.09974


![](https://raw.githubusercontent.com/jdacevedo3010/graph-mahine-learning-workshop/master/images/paper_performance.png)

The above Figure is taken from the paper, there, the authors show how Graph-Based Features Enhanced models improve the results in terms of predicting credit default over non-Grap-based features models (Base in the figure). 

Now you can try adding any graph based feature you can think of and repeat the process to see if the performance improves.

Hint: It doesn't have to be a graph Algorithm like pagerank or the degree, it can be something based on a personalized query. Think of the problem you are solving and what could add information from a graph

In [ ]:
# Feel free to use as many cells below as you need